## Введение в Машинное Обучение. 

## Практическое домашнее задание 2

### Общая информация



```
# Выбран кодовый формат
```

Дата выдачи: 08.06.2022

Дедлайн: 15.06.2022 23:59

### О задании

Задание состоит из 4 частей. В рамках данной работы вам нужно будет работать с текстовыми объявлениями с платформы Avito. 

[ссылочка на связанный контест с дз](https://www.kaggle.com/competitions/hse-intro-ds-2022-hw2/overview)

Теперь вам придется более плотнее поработать с текстом. А именно: построить некоторые статистики на имеющихся текстовых данных. Заняться лемматизацией/стэммингом, поработать с фильтрацией текстовых данных.

Данные для домашнего задания расположены там же в контесте. Он также имеет четкий дедлайн, до которого вы можете заслать туда посылки (дедлайн в нем стоит чуть позже дедлайна дз, на всякий случай).

Для доступа к данным и лидерборду в контесте необходимо принять его правила. С ними все просто.

### Формат сдачи

Оформленный юпитер ноутбук отправляете на почту: `Anshtein99@mail.ru`

В теме письма указать (x - номер группы): 2022_fikl_ml_dz2_Surname_Name

Файл должен именоваться так: dz2_Surname_Name.ipynb (и да, здесь нужны именно ваши фамилия и имя)

### Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

**Дисклеймер!!!!** не удаляйте, пожалуйста, поле id из ваших данных, ни из тестовых, ни из учебных. Иначе вашы шансы сдать свои сабмишшины в систему будут стремиться к 0

UPD. В рамках этого задания данные имеют достаточно внушительный объем, поэтому рекомендуем вам выполнять данную работу в юпитер коллабе.

Также важно!!!: промежуточные вычисления вы можете сохранять на жесткий диск коллаба, т.к. в какой-то момент все необходимые матрицы могут не влезть к вам в оперативную память.
Также стоит сохранять веса некоторых посчитанных моделей (на всякий случай)
Как это делать описано [тут](https://scikit-learn.org/stable/model_persistence.html)

### 0. Закачка данных

В рамках этой дз настоятельно рекомендуем загрузить `train.csv` и `test.csv` предварительно на ваш гугл диск. Это достаточно сильно облегчит вам жизнь при подзагрузке данных сюда в коллаб. Сами датасеты вы сможете подтянуть через код ниже:

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #путь_до_файла_с_обучением_внутри_вашего_гуглодиска

Mounted at /content/drive


In [ ]:
import pandas as pd

Train = pd.read_csv('/content/drive/MyDrive/ds_intro_hw2/train_new.csv')

Как вы можете заметить, наш датасет для обучения весит достаточно много, а в ресурсах на коллабе мы ограничены, поэтому в рамках учебного ресерча мы перейдем к исследованию конкретной части семпла нашего трейна:

In [ ]:
## ну и зафиксируем рандом на всякий случай)
Train = Train.sample(1000000, random_state=1337)
## так жизнь пойдет легче

### 1. Копаемся в полученных данных

Посмотрим на несколько строк нашего датасета. Первое, что стоит поисследовать, это наши метки, которые удобным образом представлены в текстовом формате. Поэтому...

**Заадние 1.** (1 балл). Постройте график распределения имеющихся классов. Хотелось бы увидть какую-нибудь гистограмму, где визуально видно топ-5 самых популярных и топ-5 самых непопулярных классов. Что это за категории?
Выведите из нашего трейна по паре объявлений из этих категорий. Можно ли заметить между ними (объявлениями 1 класса) какое-то сходство уже на данном этапе?

In [ ]:
# Your code here (⊃｡•́‿•̀｡)⊃

**Задание 2.** (1 балл). Попробуем также исследовать некоторое текстовое наполнение наших объявлений. Посчитайте топ-10 самых часто встречаемых слов в объявлениях (т.е. во всем 1млн строк) вне зависимости от их формы. Также рассмотрите топ-10 самых редких слов.

На данном этапе, до обработки текста, можно условиться, что за слово в объявлении мы будет считать набор символов, отделенный от других наборов символов двумя пробелами. Сделайте некоторые выводы по полученному топу и антитопу слов.

In [ ]:
# Your code here (⊃｡•́‿•̀｡)⊃

In [ ]:
Train.isnull().sum()

Видно, что есть объекты с пропуском в текстовом поле `description`. Заменим пропуски на пустую строку

In [ ]:
Train.fillna('', inplace=True)

Для простоты конкатенируем строки из полей `title` и `description`

In [ ]:
Train['title&description'] = Train['title'].str[:] + ' ' + Train['description'].str[:]

Разделим выборку ```Train``` на обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Train[['title&description']], Train['Category'], random_state=1337, test_size=100000)

del Train ## дальше в качестве данных для обучения используем X_train

### 2. Обработка текста

Как вы могли заметить в рамках предыдущих пунктов, наши объявления несут в себе некоторый текстовый "мусор". Различные местоимения, междометия, или вообще набор "\n" и "\t". Так что звучит довольно логично следующая мысль: давайте мы избавимся от этого мусора в тексте и проведем некоторую фильтрацию данных.

Также в купе с этим на ум приходит информация о том, что в наших объявлениях одни и те же слова могут быть записаны в различной языковой форме, использовать различные падежи. Поэтому что?
Да, лемматезация. Благо, мы живем с вами в современном мире, и люди для этой задачи написали уже несколько фреймворков, которые нам с вами смогут помочь в этой задаче, в этой дз я рекомендую использовать pymorphy2.

Следующая мысль, которая может прийти на ум в рамках обработки текстовых данных - это стоп слова. Их в наших объявлениях достаточно много (ведь их писали люди). Поэтому их также стоит отфильтровать.

**Задание 3.** (2 балла). Реализуйте функцию, которая принимает на вход строку, набор стоп слов и морфо-анализатор, и на выходе выдает набор слов (лемм), отчищенных от пунктуационных, разделяющих символов и стоп слов.

Некоторый её возможный шаблон представлен ниже:

Подробнее про использование pymorphy смотрите [тут](https://pymorphy2.readthedocs.io/en/0.6/user/guide.html)

In [ ]:
## блок с подтягиванием библиотеки для лемматизации
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install 'DAWG-Python >= 0.7'


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

## словарь, он же файл, который мы выдали вам вместе с домашкой, вы также можете загрузить на диск, чтобы подгрузить его в коллаб
stop_words = # Your code here (⊃｡•́‿•̀｡)⊃ пол чтению файла stop_word_ru и получения листа из нужных слов:

In [ ]:
def clear_and_morhp(text, stop_words, morph):
  '''
  text: string - строка с нашим текстом
  stop_words - List с выданными вам стоп словами
  morph - анализатор текста\слов для получения лемм
  так же не забудьте привести все слова к единому регистру
  return List слов, преобразованных до лемм + убраны различные знаки + стоп слова
  '''

  return []

  ## её использование к нашим данн в таком формате: X_train[['title&description']].apply(lambda x: clear_and_morhp(x, stop_words, morph))

Если с реализацией данной функции возникнут проблемы - не стеняйтесь писать об этом в наш чат для получения помощи.

Также вы можете реализовать не все 3 составляющих очистки текста, а какую-то их часть, но это даст вам меньше баллов.


После реализации данной функции мы можем применить её к нашим данным и получить более "чистый" набор слов.

In [ ]:
# Your code here (⊃｡•́‿•̀｡)⊃ для получения очищенных данных через вашу функцию

### 3. Построение простых эмбеддингов и работа с моделями

Сейчас вы визуально можете сравнить по определенному набору строк (допустим вызвав `.head(15)`) как изменились наши текстовые данные после предобработки. Так что мы можем с уверенностью сказать, что некоторое визуальное преобразование появилось. Но а выиграли ли мы с точки зрения достижения результата в рамках нашей задачи. Разберемся в этой части домашки.

**Задание 3.1** (0.5 балла) В рамках этого пункта нам стоит получить некоторый `бейзлайн`, с которым мы будем сравнивать остальные наши модели. Для этого вернемся к трейну X_train, в котором все еще находился мусор, слова всех форм и стоп-слова. Воспользуемся построением Tf-Idf матрицы от этих данных и также используем SGDClassifier в качестве достаточно простой модели.

Ну и в качестве метрики мы будем использовать accuracy (для подсчета этой метрики используем имеющийся у нас уже X_test), т.к. у нас все же многоклассовая классификация. Результат этой модели мы будем сравнивать с другими по ходу этого задания.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# Your code here (⊃｡•́‿•̀｡)⊃

**Задание 3.2** (0.5 балла) У метода TfidfVectorizer есть набор своих параметров. В рамках этого задания мы предлагаем вам поизучать возможные гиперпараметры этого метода и подобрать такие значения, которые вам позволят получить более лучший результат на непредобработанном X_train на той же простой модели SGDClassifier. Как вариант вы можете рассмореть н-граммы большей длины или поставить ограничение сверху на кол-во получаемых токенов в данном методе. 3-4 экспериментов с различными значениями гиперпараметорв будет достаточно, чтобы можно было сделать возможные выводы о значениях, которые подходят в рамках нашей задачи.

In [ ]:
# Your lots of code here (⊃｡•́‿•̀｡)⊃

**Задание 3.3** (3 балла) Теперь подключим рассмотрению такой подход к агрегации фичей как Bag of words. В каком-то смысле, математически он слабее tf-idf подхода, но для этого мы и столкнем их в рамках нашего задания, чтобы посмотреть на результат.

В рамках этой части домашки предобработайте **уже очищенные** данные через tf-idf и bag of words подходы. И на полученных наборах фичей обучите **SGDClassifier**, какой-нибудь градиентный бустинг (к примеру XGBoost) и удобное для вас дерево решений (из того же sklearn). 

**Минимально** у вас должно получиться **6 экспериментов**, между которыми вы сравниваете полученные accuracy. Также для устойчивого обучения стоит поиграться с параметрами двух выбранных моделей и подходов агрегации, или же провести дополнительные эксперименты, чтобы сделать выводы, как изменение параметров моделей обучения от дефолтных влияет на наши итоговые результаты.

После проведенных экспериментов стоит изложить некоторые выводы о полученных результатах. Как повлияли разные подходы tf-id и bag of words на обучение одной и той же модели, как сопоставимы результаты с этими подходами на разных моделях.

In [ ]:
# Your 4 experiments here (⊃｡•́‿•̀｡)⊃

### 4. В погоне за кагглом (снова)

**Задание 4.** (1 балл) Да, вам снова нужно попробовать собрать сабмит для нашей тестирующей системе на каггле, для этого:
Вы выбираете одну из рассмотренных пар: подхода к агрегации фичей и саму модель (или же берете вообще какой-нибудь LinearSVC, мы не запрещаем).
Делаете такую же предобработку данных к тесту (очистка, агрегация фичей), и по выбранной вами модели строите предикт для тестовых данных. Сохранить его также стоит сразу по пути на диск: `/content/drive/MyDrive/ds_intro_hw2/submit.csv`, чтобы его было оттуда файликом проще забрать.

In [ ]:
Test = pd.read_csv('/content/drive/MyDrive/то_же_самое_с_файлом_для_тестирования.csv')

# Your cleaning text and predict here (⊃｡•́‿•̀｡)⊃

### 5. Рисерч

**Задание 5.** (1 балл) В рамках этого пункта мы предлагаем вам еще поработать над данной задачей в другом подходе. В рамках задания 3 мы обучили некоторый ансамбл моделей. А что, если они будут работать над нашей задачей вместе? Да, фактически в этой задаче может помочь некотрое ансамблирование моделей.

Для его более честной и усточивой работы, к имеющимся обучающим данным вы можете применить бустреп к каждой из n-моделей (сколько вы выберете - дело ваше, но желательно не меньше 3), чтобы ей на вход поступили разные подмножества данных.

Или же вы можете поискать другие полезные закономерности в имеющихся данных, которые помогут улучшить нашу метрику качества по этой задаче. В этом плане мы вас не ограничиваем.

Но если все же в голову ничего не приходит, предлагаем рассмотреть ансамблирование нескольких моделей.

В рамках ограничения оперативной памяти, вы можете их обучить по отдельности, после сохранить черзе `pickle` и после подгрузить их для некторого ансамблирования. Итоговый класс вы можете выбирать через выбор большинства, или же предложить свою эвристику.

In [ ]:
 # Your research here (⊃｡•́‿•̀｡)⊃

### *Дополнительное задание

**Задание 6.** (+1 доп балл) Попробуйте использовать дополнительные подходы генерации признаков на текстовых даннных (fasttext, w2v и др) и для дальнейшего обучения моделей. Повысилось ли качество модели? Сделайте выводы.


In [ ]:
 # Your research here (⊃｡•́‿•̀｡)⊃